# Earning Prediction

### Libraries
- https://pandas.pydata.org/
- http://scikit-learn.org/stable/
- https://keras.io/
- https://www.tensorflow.org/

### Source
- https://www.linkedin.com/learning/building-deep-learning-applications-with-keras-2-0

In [40]:
# Configuration

RUN_NAME = "run 2 with 5 nodes"

In [48]:
# Libraries

import pandas as pd
from sklearn.preprocessing import MinMaxScaler

import keras
from keras.models import Sequential
from keras.layers import *
from keras.models import load_model

import tensorflow as tf

In [37]:
# Load and transform data

# Load training data set from CSV file
training_data_df = pd.read_csv("data/sales_data_training.csv")

# Load testing data set from CSV file
test_data_df = pd.read_csv("data/sales_data_test.csv")

# Data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well.
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
scaled_training = scaler.fit_transform(training_data_df)
scaled_testing = scaler.transform(test_data_df)

# Print out the adjustment that the scaler applied to the total_earnings column of data
print("Note: total_earnings values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[8], scaler.min_[8]))

# Create new pandas DataFrame objects from the scaled data
scaled_training_df = pd.DataFrame(scaled_training, columns=training_data_df.columns.values)
scaled_testing_df = pd.DataFrame(scaled_testing, columns=test_data_df.columns.values)

# Save scaled data dataframes to new CSV files
scaled_training_df.to_csv("data/sales_data_training_scaled.csv", index=False)
scaled_testing_df.to_csv("data/sales_data_testing_scaled.csv", index=False)

Note: total_earnings values were scaled by multiplying by 0.0000036968 and adding -0.115913


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [42]:
# Build the model

scaled_training_df = pd.read_csv("data/sales_data_training_scaled.csv")

X = scaled_training_df.drop('total_earnings', axis=1).values
Y = scaled_training_df[['total_earnings']].values

# Define the model
model = Sequential()
model.add(Dense(5, input_dim=9, activation='relu', name='layer_1'))
model.add(Dense(100, activation='relu', name='layer_2'))
model.add(Dense(50, activation='relu', name='layer_3'))
model.add(Dense(1, activation='linear', name='output_layer'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [44]:
# Train the model

# Create a TensorBoard logger
logger = keras.callbacks.TensorBoard(
    log_dir='logs/{}'.format(RUN_NAME),
    histogram_freq=0,
    write_graph=True
)

# Train the model
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2,
    callbacks=[logger]
)

# Load the separate test data set
scaled_testing_df = pd.read_csv("data/sales_data_testing_scaled.csv")

X_test = scaled_testing_df.drop('total_earnings', axis=1).values
Y_test = scaled_testing_df[['total_earnings']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

# Save the model to disk
model.save("models/trained_model.h5")
print("Model saved to disk.")

Epoch 1/50
 - 0s - loss: 0.0234
Epoch 2/50
 - 0s - loss: 0.0204
Epoch 3/50
 - 0s - loss: 0.0182
Epoch 4/50
 - 0s - loss: 0.0164
Epoch 5/50
 - 0s - loss: 0.0140
Epoch 6/50
 - 0s - loss: 0.0116
Epoch 7/50
 - 0s - loss: 0.0092
Epoch 8/50
 - 0s - loss: 0.0071
Epoch 9/50
 - 0s - loss: 0.0051
Epoch 10/50
 - 0s - loss: 0.0036
Epoch 11/50
 - 0s - loss: 0.0027
Epoch 12/50
 - 0s - loss: 0.0019
Epoch 13/50
 - 0s - loss: 0.0015
Epoch 14/50
 - 0s - loss: 0.0013
Epoch 15/50
 - 0s - loss: 0.0010
Epoch 16/50
 - 0s - loss: 8.8284e-04
Epoch 17/50
 - 0s - loss: 7.6855e-04
Epoch 18/50
 - 0s - loss: 6.7854e-04
Epoch 19/50
 - 0s - loss: 5.7385e-04
Epoch 20/50
 - 0s - loss: 5.0943e-04
Epoch 21/50
 - 0s - loss: 4.3227e-04
Epoch 22/50
 - 0s - loss: 3.8155e-04
Epoch 23/50
 - 0s - loss: 3.5938e-04
Epoch 24/50
 - 0s - loss: 3.2540e-04
Epoch 25/50
 - 0s - loss: 3.4207e-04
Epoch 26/50
 - 0s - loss: 2.8339e-04
Epoch 27/50
 - 0s - loss: 2.4162e-04
Epoch 28/50
 - 0s - loss: 2.1254e-04
Epoch 29/50
 - 0s - loss: 1.8300e

In [46]:
# Predict

model = load_model('models/trained_model.h5')

X = pd.read_csv("data/proposed_new_product.csv").values
prediction = model.predict(X)

# Grab just the first element of the first prediction (since we only have one)
prediction = prediction[0][0]

# Re-scale the data from the 0-to-1 range back to dollars
# These constants are from when the data was originally scaled down to the 0-to-1 range
prediction = prediction - scaler.min_[8]
prediction = prediction / scaler.scale_[8]

print("Earnings Prediction for Proposed Product - ${}".format(prediction))

Earnings Prediction for Proposed Product - $255985.67072808743


In [49]:
# Export model

model_builder = tf.saved_model.builder.SavedModelBuilder("models/exported_model")

inputs = {
    'input': tf.saved_model.utils.build_tensor_info(model.input)
}
outputs = {
    'earnings': tf.saved_model.utils.build_tensor_info(model.output)
}

signature_def = tf.saved_model.signature_def_utils.build_signature_def(
    inputs=inputs,
    outputs=outputs,
    method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
)

model_builder.add_meta_graph_and_variables(
    K.get_session(),
    tags=[tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_def
    }
)

model_builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: models/exported_model/saved_model.pb


b'models/exported_model/saved_model.pb'

In [ ]:
# predict from Google cloud

from oauth2client.client import GoogleCredentials
import googleapiclient.discovery

# Change this values to match your project
PROJECT_ID = "keras-class"
MODEL_NAME = "earnings"
CREDENTIALS_FILE = "credentials.json"

# These are the values we want a prediction for
inputs_for_prediction = [
    {"input": [0.4999, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.5]}
]

# Connect to the Google Cloud-ML Service
credentials = GoogleCredentials.from_stream(CREDENTIALS_FILE)
service = googleapiclient.discovery.build('ml', 'v1', credentials=credentials)

# Connect to our Prediction Model
name = 'projects/{}/models/{}'.format(PROJECT_ID, MODEL_NAME)
response = service.projects().predict(
    name=name,
    body={'instances': inputs_for_prediction}
).execute()

# Report any errors
if 'error' in response:
    raise RuntimeError(response['error'])

# Grab the results from the response object
results = response['predictions']

# Print the results!
print(results)
